In [121]:
import pandas as pd

df = pd.read_csv('data_4.csv')

df.head()
# df["clean_text"] = df["fixed_text"] # only for data_1

,category,text,clean_text
0,arts,you are using an older browser version please ...,older browser version please use version best ...
1,arts,on march two of classical music s most accompl...,march two classical music accomplished well kn...
2,arts,bpt after a year of being locked away at home ...,year locked away home world eager reopen exper...
3,arts,pilot uninjured plane hit sandbar while landin...,pilot uninjured plane hit landing float plane ...
4,arts,colleen distin photo by facebook toronto sun a...,colleen photo sun lost wallet returned owner c...


In [77]:
# from nltk import pos_tag
# from collections import Counter
# import nltk
# from tqdm import tqdm
# tqdm.pandas()
# 
# nltk.download('averaged_perceptron_tagger_eng')
# 
# df["text_stopwords_removed"] = df["text_stopwords_removed"].apply(lambda x: x if isinstance(x, list) else nltk.word_tokenize(x))
# 
# df["pos_text"] = df["text_stopwords_removed"].progress_apply(lambda x: Counter(tag for _, tag in pos_tag(x)))

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/karennurlybekov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
100%|██████████| 5292/5292 [01:41<00:00, 52.17it/s]


In [90]:
df.head()

,category,text,clean_text,label_encoded
0,crime,police gather at the publix shopping center at...,police gather shopping center scene shooting r...,1
1,economy,news feed neighbor posts classifieds calendar ...,news feed neighbor calendar rancho santa marga...,3
2,disaster,bicyclist meets paramedics who saved him after...,bicyclist saved crash matt come heart worked b...,2
3,other,kabul afghanistan ap a helicopter crash overni...,helicopter crash overnight taken least nine af...,10
4,weather,london paris moody s esg solutions announced t...,paris moody today provided second party opinio...,17


In [122]:
import spacy
from tqdm import tqdm

tqdm.pandas()
nlp = spacy.load("en_core_web_sm")


def get_pos_counts(doc_text):
    """
    Parse the text with spaCy, count occurrences of chosen PoS types.
    Returns a dict of PoS counts.
    """
    doc = nlp(doc_text)
    
    # Initialize counters
    noun_count = 0   # spaCy pos_ == "NOUN"
    verb_count = 0   # "VERB"
    adj_count = 0    # "ADJ"
    adv_count = 0    # "ADV"
    propn_count = 0  # "PROPN" (proper noun)
    
    for token in doc:
        if token.pos_ == "NOUN":
            noun_count += 1
        elif token.pos_ == "VERB":
            verb_count += 1
        elif token.pos_ == "ADJ":
            adj_count += 1
        elif token.pos_ == "ADV":
            adv_count += 1
        elif token.pos_ == "PROPN":
            propn_count += 1
    
    return {
        "noun_count": noun_count,
        "verb_count": verb_count,
        "adj_count": adj_count,
        "adv_count": adv_count,
        "propn_count": propn_count
    }

# Apply the function to "clean_text"
pos_features_series = df["clean_text"].progress_apply(get_pos_counts)
# Convert the series of dictionaries into a DataFrame
pos_features_df = pd.DataFrame(pos_features_series.tolist())

# Merge PoS features back into the main df if you want
df_pos = pd.concat([df, pos_features_df], axis=1)

df_pos.head()

100%|██████████| 5292/5292 [01:30<00:00, 58.30it/s]


,category,text,clean_text,noun_count,verb_count,adj_count,adv_count,propn_count
0,arts,you are using an older browser version please ...,older browser version please use version best ...,53,24,28,5,15
1,arts,on march two of classical music s most accompl...,march two classical music accomplished well kn...,149,51,81,45,44
2,arts,bpt after a year of being locked away at home ...,year locked away home world eager reopen exper...,70,44,52,20,40
3,arts,pilot uninjured plane hit sandbar while landin...,pilot uninjured plane hit landing float plane ...,25,7,3,0,8
4,arts,colleen distin photo by facebook toronto sun a...,colleen photo sun lost wallet returned owner c...,55,20,12,9,19


In [123]:
df['category'].value_counts()

category
science          299
disaster         297
crime            297
weather          296
education        296
social           295
humaninterest    295
religion         295
health           294
politics         294
economy          293
unrest           293
labour           293
sport            293
arts             291
environmental    291
lifestyle        291
other            289
Name: count, dtype: int64

In [ ]:
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.metrics import accuracy_score
# 
# 
# X = df["clean_text"]  # after your cleaning steps
# y = df["category"]
# 
# # 2. Train-Test Split
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.1, 
#                                                     random_state=42)
# 
# # 3. Vectorize
# # tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', min_df=2)
# tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
# 
# X_train_tfidf = tfidf.fit_transform(X_train)
# X_test_tfidf = tfidf.transform(X_test)
# 
# # 4. Optional: Feature Selection
# k = 2000  # or pick another number
# selector = SelectKBest(chi2, k=k)
# X_train_selected = selector.fit_transform(X_train_tfidf, y_train)
# X_test_selected = selector.transform(X_test_tfidf)
# 
# # 5. Model Tuning
# param_dist = {
#     'n_estimators': [100, 200, 300, 400],
#     'max_depth': [10, 20, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt', 'log2', 0.5]
# }
# 
# rf = RandomForestClassifier(random_state=42, class_weight='balanced')
# 
# random_search = RandomizedSearchCV(
#     estimator=rf,
#     param_distributions=param_dist,
#     n_iter=20,
#     scoring='accuracy',
#     cv=5,
#     random_state=42,
#     n_jobs=-1
# )
# 
# random_search.fit(X_train_selected, y_train) # or X_train_tfidf for whole X_train_selected for k-best
# best_rf = random_search.best_estimator_
# 
# # 6. Evaluate
# y_pred = best_rf.predict(X_test_selected)  # or X_test_tfidf for whole or X_test_selected for k-best
# acc = accuracy_score(y_test, y_pred)
# print("Best params:", random_search.best_params_)
# print("Test Accuracy:", acc)

In [142]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 1. Load Data
# df should have columns: "clean_text", "category"
X = df["clean_text"]
y = df["category"]

# 2. Train/Test Split (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.1, 
    random_state=42, 
    stratify=y
)

# 3. Define Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 2),
        stop_words='english'
    )),
    ('selector', SelectKBest(chi2, k=5000)),  # default k=5000 (will be tuned)
    ('clf', RandomForestClassifier(
        criterion='gini',
        class_weight='balanced',
        bootstrap=True,
        n_jobs=-1,
        random_state=42
    ))
])

# 4. Define Hyperparameter Search Space
param_dist = {
    'selector__k': [1000, 2000, 3000, 5000],
    'clf__n_estimators': [100, 200, 300, 400],
    'clf__max_depth': [10, 20, 30, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['sqrt', 'log2', 0.5]
}

# 5. Cross-validation Setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 6. RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=20,              # number of parameter settings sampled
    scoring='accuracy',     # or 'f1_macro', etc.
    cv=skf,
    random_state=42,
    n_jobs=-1
)

# 7. Fit the Pipeline
random_search.fit(X_train, y_train)

# 8. Retrieve Best Model and Evaluate
best_model = random_search.best_estimator_
print("Best Params:", random_search.best_params_)

y_pred = best_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='macro')
rec = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Test Accuracy:      {acc:.4f}")
print(f"Macro Precision:    {prec:.4f}")
print(f"Macro Recall:       {rec:.4f}")
print(f"Macro F1 Score:     {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Params: {'selector__k': 2000, 'clf__n_estimators': 300, 'clf__min_samples_split': 10, 'clf__min_samples_leaf': 1, 'clf__max_features': 'log2', 'clf__max_depth': None}
Test Accuracy:      0.5340
Macro Precision:    0.5261
Macro Recall:       0.5335
Macro F1 Score:     0.5085

Classification Report:
               precision    recall  f1-score   support

         arts       0.50      0.45      0.47        29
        crime       0.43      0.67      0.53        30
     disaster       0.63      0.40      0.49        30
      economy       0.54      0.52      0.53        29
    education       0.60      0.87      0.71        30
environmental       0.76      0.76      0.76        29
       health       0.51      0.86      0.64        29
humaninterest       0.45      0.30      0.36        30
       labour       0.42      0.45      0.43        29
    lifestyle       0.42      0.52      0.46        29
        other       0.25      0.03      0.06        29
     politics       0.39      0.24 

In [125]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_pos["label_encoded"] = label_encoder.fit_transform(df["category"])

df_pos.head()

,category,text,clean_text,noun_count,verb_count,adj_count,adv_count,propn_count,label_encoded
0,arts,you are using an older browser version please ...,older browser version please use version best ...,53,24,28,5,15,0
1,arts,on march two of classical music s most accompl...,march two classical music accomplished well kn...,149,51,81,45,44,0
2,arts,bpt after a year of being locked away at home ...,year locked away home world eager reopen exper...,70,44,52,20,40,0
3,arts,pilot uninjured plane hit sandbar while landin...,pilot uninjured plane hit landing float plane ...,25,7,3,0,8,0
4,arts,colleen distin photo by facebook toronto sun a...,colleen photo sun lost wallet returned owner c...,55,20,12,9,19,0


In [86]:
# import nltk
# from nltk.tokenize import word_tokenize
# 
# nltk.download("punkt")
# 
# df["tokenized_text"] = df["text"].apply(lambda x: word_tokenize(x.lower()))
# 
# df.head()
# pos_features_df = pd.DataFrame(df["pos_text"].tolist()).fillna(0)
# 
# df = df.drop(columns=["pos_text"])  # Remove the raw PoS column
# df = pd.concat([df, pos_features_df], axis=1)  # Merge PoS features

df.head()

,category,text,clean_text,label_encoded
0,crime,police gather at the publix shopping center at...,police gather shopping center scene shooting r...,1
1,economy,news feed neighbor posts classifieds calendar ...,news feed neighbor calendar rancho santa marga...,3
2,disaster,bicyclist meets paramedics who saved him after...,bicyclist saved crash matt come heart worked b...,2
3,other,kabul afghanistan ap a helicopter crash overni...,helicopter crash overnight taken least nine af...,10
4,weather,london paris moody s esg solutions announced t...,paris moody today provided second party opinio...,17


In [126]:
from sklearn.decomposition import PCA
import numpy as np
import spacy

# tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)  
# X_tfidf_sparse = tfidf_vectorizer.fit_transform(df["clean_text"])  
# # For combining with dense arrays, convert to dense
# X_tfidf = X_tfidf_sparse.toarray()  # shape: (n_samples, vocab_size)
# 
# nlp = spacy.load("en_core_web_md")
# 
# def get_word_embeddings_spacy(text):
#     doc = nlp(text)
#     # doc.vector is the average of token vectors (300D for en_core_web_md)
#     return doc.vector
# 
# # Apply to each row
# df["embeddings"] = df["clean_text"].apply(get_word_embeddings_spacy)
# # Convert list of vectors into a 2D numpy array
# X_embeddings = np.vstack(df["embeddings"].values)  # shape: (n_samples, 300)
# 
# # Reduce dimensionality from 300 to 50 (example)
# pca = PCA(n_components=50, random_state=42)
# X_embeddings_reduced = pca.fit_transform(X_embeddings)  # shape: (n_samples, 50)
# 
# # Optional: Drop "embeddings" column
# df.drop(columns=["embeddings"], inplace=True)




# X_combined = np.hstack((X_tfidf, X_embeddings_reduced))
# print("X_combined shape:", X_combined.shape)



vectorizer = TfidfVectorizer()
X_tfidf_sparse = vectorizer.fit_transform(df_pos["clean_text"])  # shape: (n_samples, vocab_size)

# If you plan to combine with numeric PoS features, you'll often need a dense array:
X_tfidf = X_tfidf_sparse.toarray()  # shape: (n_samples, vocab_size)

# -----------------------------------------------------------------------------
# 6) Combine TF-IDF and PoS Features
# -----------------------------------------------------------------------------
import numpy as np

# Extract PoS columns as a NumPy array
X_pos = df_pos[["noun_count", "verb_count", "adj_count", "adv_count", "propn_count"]].values
# shape: (n_samples, 5)

# hstack them horizontally
X_combined = np.hstack((X_tfidf, X_pos))
# shape: (n_samples, vocab_size + 5)

# Labels
y = df_pos["label_encoded"].values

X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=42
)

clf = RandomForestClassifier(n_estimators=500, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


Test Accuracy: 0.5004721435316336


In [69]:
# import numpy as np
# import spacy
# nlp = spacy.load("en_core_web_md")  # Medium model with word embeddings
# 
# def get_word_embeddings(text):
#     """Extracts a 300-dimensional word vector from text using spaCy"""
#     if isinstance(text, list):  # If tokenized, convert back to string
#         text = " ".join(text)
#     doc = nlp(text)
#     return doc.vector  # 300-dim vector
# 
# # Apply word embeddings to text column
# df["word_embeddings"] = df["text_processed"].apply(get_word_embeddings)
# 
# # Convert list of arrays into a NumPy array
# X_embeddings = np.vstack(df["word_embeddings"].values)  
# 
# # Drop the temporary embeddings column
# df.drop(columns=["word_embeddings"], inplace=True)
# 
# # Convert PoS features to NumPy array
# X_pos = pos_features_df.values  
# 
# # Merge Embeddings + PoS Features
# X_combined = np.hstack((X_embeddings, X_pos))
# 
# # Define labels
# y = df["label_encoded"].values  # Ensure it's numerical (use LabelEncoder if needed)


# df.drop(columns=["text", "tokenized_text","text_stopwords_removed","fixed_text"], inplace=True)
# df.head()

,category,label_encoded,RB,VBG,'',",",NNP,POS,NN,IN,...,DT,FW,EX,VBN,JJR,VBP,CC,WP,VB,text_processed
0,arts,0,7.0,11.0,88,183,96,120,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'using ' , 'older ' , 'browser ' , 'version ..."
1,arts,0,28.0,45.0,323,558,239,311,23.0,6.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'march ' , 'two ' , 'classical ' , 'music ' ..."
2,arts,0,3.0,21.0,182,343,165,216,7.0,8.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'bpt ' , 'year ' , 'locked ' , 'away ' , 'ho..."
3,arts,0,0.0,6.0,30,75,45,52,3.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'pilot ' , 'uninjured ' , 'plane ' , 'hit ' ..."
4,arts,0,1.0,5.0,81,171,91,113,8.0,7.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'colleen ' , 'distin ' , 'photo ' , 'faceboo..."


In [127]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=500, max_depth=20, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, classification_report


# Predict on test data
y_pred = rf.predict(X_test)

# Decode predicted labels back to text
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test_labels, y_pred_labels)

# Display results
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

# Show the first few rows with predictions
df_test_results = pd.DataFrame({"text_processed": df["text_processed"].iloc[y_test.index], "actual": y_test_labels, "predicted": y_pred_labels})

Accuracy: 0.49
Classification Report:
                precision    recall  f1-score   support

         arts       0.55      0.41      0.47        66
        crime       0.42      0.48      0.45        61
     disaster       0.74      0.40      0.52        70
      economy       0.33      0.36      0.34        56
    education       0.53      0.79      0.63        61
environmental       0.63      0.69      0.66        65
       health       0.34      0.75      0.46        51
humaninterest       0.55      0.32      0.41        68
       labour       0.56      0.67      0.61        55
    lifestyle       0.45      0.35      0.39        55
        other       0.33      0.05      0.08        44
     politics       0.59      0.25      0.35        64
     religion       0.47      0.63      0.54        57
      science       0.53      0.47      0.50        62
       social       0.83      0.15      0.26        65
        sport       0.36      0.70      0.48        53
       unrest       0.54 

KeyError: 'text_processed'

In [44]:
df.head()

,category,text,tokenized_text,text_stopwords_removed,fixed_text,label_encoded,RB,VBG,'',",",...,DT,FW,EX,VBN,JJR,VBP,CC,WP,VB,text_processed
0,arts,you are using an older browser version please ...,"['using', 'older', 'brother', 'version', 'plea...","[[, 'using, ', ,, 'older, ', ,, 'browser, ', ,...",using older brother version please use support...,0,7.0,11.0,88,183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'using ' , 'older ' , 'browser ' , 'version ..."
1,arts,on march two of classical music s most accompl...,"['march', 'two', 'classical', 'music', 'accomp...","[[, 'march, ', ,, 'two, ', ,, 'classical, ', ,...",march two classical music accomplished well kn...,0,28.0,45.0,323,558,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'march ' , 'two ' , 'classical ' , 'music ' ..."
2,arts,bpt after a year of being locked away at home ...,"['but', 'year', 'locked', 'away', 'home', 'wor...","[[, 'bpt, ', ,, 'year, ', ,, 'locked, ', ,, 'a...",but year locked away home world eager open exp...,0,3.0,21.0,182,343,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'bpt ' , 'year ' , 'locked ' , 'away ' , 'ho..."
3,arts,pilot uninjured plane hit sandbar while landin...,"['pilot', 'uninjured', 'plane', 'hit', 'sandba...","[[, 'pilot, ', ,, 'uninjured, ', ,, 'plane, ',...",pilot uninjured plane hit sandbag landing floa...,0,0.0,6.0,30,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'pilot ' , 'uninjured ' , 'plane ' , 'hit ' ..."
4,arts,colleen distin photo by facebook toronto sun a...,"['college', 'listen', 'photo', 'facebook', 'to...","[[, 'colleen, ', ,, 'distin, ', ,, 'photo, ', ...",college listen photo facebook toronto sun lost...,0,1.0,5.0,81,171,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[ 'colleen ' , 'distin ' , 'photo ' , 'faceboo..."


In [45]:
print(df["label_encoded"].value_counts(normalize=True))


label_encoded
13    0.056500
2     0.056122
1     0.056122
17    0.055933
4     0.055933
14    0.055745
7     0.055745
12    0.055745
6     0.055556
11    0.055556
3     0.055367
16    0.055367
8     0.055367
15    0.055367
0     0.054989
5     0.054989
9     0.054989
10    0.054611
Name: proportion, dtype: float64
